In [1]:
# Imports
import random

import math

class Vertex:
    def __init__(self, key, x, y):
        self.key = key
        self.x = x
        self.y = y
        self.d = float('-inf')  # Distance from source
        self.h = None  # Heuristic distance to destination
        self.pi = None
        self.pi_set = set()
        self.priority = 0
        self.parent = None

    def __lt__(self, other):
        return self.priority < other.priority

    def __gt__(self, other):
        return self.priority > other.priority


class Graph:
    def __init__(self):
        self.vertices = {}
        self.adjacency_list = {}

    def add_vertex(self, key, x, y):
        if key not in self.vertices:
            self.vertices[key] = Vertex(key, x, y)
            self.adjacency_list[key] = []

    def add_edge(self, u, v):
        if u in self.vertices and v in self.vertices and u != v:
            if v not in self.adjacency_list[u]:
                self.adjacency_list[u].append(v)
            if u not in self.adjacency_list[v]:
                self.adjacency_list[v].append(u)  # For undirected graph


    def get_adjacency_list(self, vertex_key):
        if vertex_key in self.vertices:
            return self.adjacency_list[vertex_key]
        else:
            return []
    
    def max_degree(self):
        max_degree = 0
        for vertex_key in self.vertices.keys():
            degree = len(self.adjacency_list[vertex_key])
            max_degree = max(max_degree, degree)
        return max_degree
    
    def average_degree(self):
        total_degree = 0
        for vertex_key in self.vertices.keys():
            total_degree += len(self.adjacency_list[vertex_key])
        if len(self.vertices) > 0:
            return total_degree / len(self.vertices)
        else:
            return 0

    def generate_geometric_graph(self, n, r):

        for key in range(n):
            self.add_vertex(key,random.uniform(0, 1), random.uniform(0, 1))

        for u in self.vertices:
            for v in self.vertices:
                if u != v and euclidean_distance(self.vertices[u], self.vertices[v]) <= r:
                    self.adjacency_list[u].append(v)
                    self.adjacency_list[v].append(u)


def euclidean_distance(v1, v2):
    return math.sqrt(((v1.x - v2.x) ** 2) + ((v1.y - v2.y) ** 2))



n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

graph = Graph()

def largest_connected_component(graph):
    def dfs(graph, visited, vertex, component):
        visited.add(vertex)
        component.append(vertex)
        for neighbor in graph.get_adjacency_list(vertex):
            if neighbor not in visited:
                dfs(graph, visited, neighbor, component)

    visited = set()
    largest_component = []
    for vertex in graph.vertices:
        if vertex not in visited:
            component = []
            dfs(graph, visited, vertex, component)
            if len(component) > len(largest_component):
                largest_component = component

    lcc_graph = Graph()
    for vertex_key in largest_component:
        vertex = graph.vertices[vertex_key]
        lcc_graph.add_vertex(vertex.key, vertex.x, vertex.y)
        for neighbor_key in graph.get_adjacency_list(vertex_key):
            if neighbor_key in largest_component:
                lcc_graph.add_edge(vertex.key, neighbor_key)

    return lcc_graph


def binary_search_r(n, min_ratio, max_ratio):
    low = 0.0
    high = 1.0
    while low <= high:
        graph = Graph()
        mid = (low + high) / 2
        vlcc_min = int(min_ratio * n)
        vlcc_max = int(max_ratio * n)
        graph.generate_geometric_graph(n, mid)
        lcc = largest_connected_component(graph)
        lcc_size = len(lcc.vertices)
        if vlcc_min <= lcc_size <= vlcc_max:
            return mid, lcc
        elif lcc_size > vlcc_min:
            high = mid - 0.00001
        else:
            low = mid + 0.00001
    return None, None  # No suitable r found


def store_graph_to_file(graph, filename):
    with open(filename, 'w') as f:
        processed_edges = set()  # To avoid duplicates
        for u in graph.vertices:
            for v in graph.get_adjacency_list(u):
                # Generate a unique key for the edge (u, v) by sorting the vertices' keys
                edge_key = (u, v) if u < v else (v, u)
                if edge_key not in processed_edges:
                    vertex_u = graph.vertices[u]
                    vertex_v = graph.vertices[v]
                    f.write(f"{u} {vertex_u.x} {vertex_u.y} {v} {vertex_v.x} {vertex_v.y}\n")
                    processed_edges.add(edge_key)


n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

# for n, min_ratio, max_ratio in zip(n_values, min_ratios, max_ratios):
#     r, lcc = binary_search_r(n, min_ratio, max_ratio)
#     if r is not None:
#         store_graph_to_file(lcc, f"graph_n{n}_r{r}.EDGES")
#         print(f"Graph with n={n} and r={r} stored successfully.")
#     else:
#         print(f"No suitable r found for n={n}.")


def read_graph_from_file(filename):
    graph = Graph()
    with open(filename, 'r') as f:
        for line in f:
            data = line.split()
            u = int(data[0])
            x1 = float(data[1])
            y1 = float(data[2])
            v = int(data[3])
            x2 = float(data[4])
            y2 = float(data[5])

            if u not in graph.vertices:
                graph.add_vertex(u, x1, y1)
            if v not in graph.vertices:
                graph.add_vertex(v, x2, y2)

            graph.add_edge(u, v)

    return graph


graph = read_graph_from_file("graph_n300_r0.08202289062500001.EDGES")
# graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")
# graph = read_graph_from_file("graph_n500_r0.05467859375.EDGES")

if 13 in graph.vertices:
    print("yes")


In [2]:
len(graph.vertices)

282

In [6]:
def DepthLimitedDFS(graph, node, visited, path, max_depth, current_depth, max_length, longest_path):
    visited[node] = True
    path.append(node)

    if current_depth == max_depth:
        if len(path) > max_length[0]:
            max_length[0] = len(path)
            longest_path.clear()
            longest_path.extend(path)
    else:
        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                DepthLimitedDFS(graph, neighbor, visited, path, max_depth, current_depth + 1, max_length, longest_path)

    path.pop()
    visited[node] = False

def IterativeDeepeningDFS(graph):
    max_length = [float('-inf')]
    longest_path = []
    visited = {v: False for v in graph.vertices}

    for depth in range(len(graph.vertices)):
        for vertex in graph.vertices:
            DepthLimitedDFS(graph, vertex, visited, [], depth, 0, max_length, longest_path)

    return longest_path

print(sorted(IterativeDeepeningDFS(graph)))

In [4]:
def DFSLongestPath(graph):
    def dfs(node, path, memo):
        if node in memo:
            return memo[node]
        
        longest = path + [node]
        for neighbor in graph.adjacency_list[node]:
            if neighbor not in path:
                candidate = dfs(neighbor, path + [node], memo)
                if len(candidate) > len(longest):
                    longest = candidate

        memo[node] = longest
        return longest

    memo = {}
    longest_path = []

    for vertex in graph.vertices:
        current_path = dfs(vertex, [], memo)
        if len(current_path) > len(longest_path):
            longest_path = current_path

    return longest_path

# Example usage:
longest_path = DFSLongestPath(graph)
# print("Longest path:", sorted(longest_path))
longest_path


[0,
 9,
 80,
 91,
 32,
 116,
 177,
 30,
 4,
 44,
 16,
 96,
 271,
 70,
 172,
 185,
 291,
 279,
 102,
 21,
 15,
 31,
 36,
 100,
 140,
 107,
 59,
 112,
 95,
 143,
 88,
 117,
 61,
 152,
 7,
 47,
 34,
 56,
 83,
 219,
 218,
 111,
 2,
 29,
 128,
 46,
 79,
 27,
 191,
 273,
 216,
 293,
 120,
 43,
 97,
 270,
 50,
 41,
 104,
 58,
 19,
 267,
 11,
 24,
 63,
 98,
 214,
 155,
 182,
 25,
 5,
 123,
 205,
 252,
 101,
 121,
 255,
 180,
 160,
 106,
 165,
 264,
 53,
 105,
 139,
 72,
 110,
 51,
 20,
 55,
 158,
 6,
 35,
 8,
 130,
 86,
 28,
 45,
 151,
 237,
 262,
 89,
 94,
 223,
 217,
 87,
 60,
 114,
 161,
 147,
 12,
 149,
 118,
 14,
 108,
 127,
 170,
 297,
 184,
 148,
 163]